In [5]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *
# Import du paquet time pour calculer le temps de résolution
import time 
import InstanceReader

In [49]:
def solve(data, L, K):
    # Création du modèle vide 
    model = Model(name = "MTZ-EAF", sense = mip.MAXIMIZE, solver_name="CBC")
    # Création des variables y, u et t
    y=[]
    u=[]
    #y = [[model.add_var(name="y(" + str(data.A[i][j]) + "," + str(data.A[i][j+1]) + ")", var_type=BINARY) for i in range(len(data.A))] for j in range(1)]
    for i in (data.V):
        for j in (data.allPatients[int(data.index[int(i)])]):
            y.append(model.add_var(name="y(" + str(i) + "," + str(j) + ")", var_type=BINARY))
    for k in range(len(data.P)):
        for i in (data.P):
            for j in (data.allPatients[int(data.index[int(i)])]):
                u.append(model.add_var(name="u("+str(k) + "," + str(i) + "," + str(j)+ ")", var_type=BINARY))
    t = [model.add_var(name="t(" + str(i)  + ")", lb=0, ub=(L-1) , var_type=CONTINUOUS) for i in range(len(data.V))]
        
    
    # Ajout de la fonction objectif au modèle
    # !!! fonction non valide !!!
    #deuxième partie de la somme (fausse aussi) en dessous
    model += (xsum(data.cost[i][j] * y[i][j]  for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])]) ))
    # !!! fonction non valide !!! 
    
    #Les contraintes
    for k in range(len(data.P)):
        for i in (data.P):
            model += (xsum(u[k][i][j] for j in data.allPatients[int(data.index[int(i)])])+ xsum(u[k][int(data.allDonors[i][j])][i] for j in range(1,data.allDonors[i][0])) ==0 ) 
    
    
    
    # Limitation du nombre de processeurs
    model.threads = 1  
    # Lancement du chronomètre
    start = time.perf_counter()
    # Résolution du modèle
    status = model.optimize(max_seconds=120)  # temps limite = 120s
    # Arrêt du chronomètre et calcul du temps de résolution
    runtime = time.perf_counter() - start
    
    print("\n----------------------------------")
    if status == OptimizationStatus.OPTIMAL:
        print("Status de la résolution: OPTIMAL")
    elif status == OptimizationStatus.FEASIBLE:
        print("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
    elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
        print("Status de la résolution: IRREALISABLE")
    elif status == OptimizationStatus.UNBOUNDED:
        print("Status de la résolution: NON BORNE")
        
    print("Temps de résolution (s) : ", runtime)    
    print("----------------------------------")

    # Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
    if model.num_solutions>0:
        print("Valeur de la fonction objectif de la solution calculée : ", round(model.objective_value,2))  # ne pas oublier d'arrondir si le coût doit être entier
        
    

In [50]:
solve(InstanceReader.Data(str(50),str(0)),4,4)

id max :  127



TypeError: 'Var' object is not subscriptable

In [ ]:
+ xsum(xsum(data.cost[data.A[i][0]][data.A[i][1]]*u[k][data.A[i][0]][data.A[i][1]] for i in range(len(data.A)))for k in range (len(data.P)))